In [ ]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

import os


## Kernel Density Estimate Plots

In [ ]:
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df,label_name):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df[label_name].corr(df[var_name])
            
    plt.figure(figsize = (12, 6))
    for value in df[label_name].unique():
        # Plot the distribution for target == 0 and target == 1
        sns.kdeplot(df.ix[df[label_name] == value, var_name], label = 'Label : '+ str(value))

    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    for value in df[label_name].unique():
        # Calculate medians for repaid vs not repaid
        avg_label_val = df.ix[df[label_name] == value, var_name].median()
        print('Median value for {%s} =     %0.4f' %(value ,avg_label_val))
    # print out the correlation
    print('The correlation between %s and the Target variable is %0.4f' % (var_name, corr))
    

## Adding aggregated features

In [ ]:
# Group by some feature, calculate aggregation statistics
feature= "<insert some feature>" 
df_agg = df.drop(columns = ['']).groupby(feature, as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
df_agg.head()

In [ ]:
# Iterate through the variables names
for var in bureau_agg.columns.levels[0]:
    # Skip the id name
    if var != feature:
        
        # Iterate through the stat names
        for stat in df_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('df_%s_%s' % (var, stat))

In [ ]:
#List of new correlations
new_corrs = []

# Iterate through the columns 
for col in columns:
    # Calculate correlation with the target
    corr = df[label_name].corr(df[col])
    
    # Append the list as a tuple

    new_corrs.append((col, corr))

In [ ]:
# Rank them in reverse order of the absolute correlation values and  
# use the features that qualify above a given threshold
new_corrs = sorted(new_corrs,key=lambda x :-abs(x[1]))

In [ ]:
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg

In [ ]:
# Function to calculate correlations with the target for a dataframe
def target_corrs(df,label_name):

    # List of correlations
    corrs = []

    # Iterate through the columns 
    for col in df.columns:
        print(col)
        # Skip the target column
        if col != label_name:
            # Calculate correlation with the target
            corr = df[label_name].corr(df[col])

            # Append the list as a tuple
            corrs.append((col, corr))
            
    # Sort by absolute magnitude of correlations
    corrs = sorted(corrs, key = lambda x: abs(x[1]), reverse = True)
    
    return corrs